In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
'''
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
'''
# Any results you write to the current directory are saved as output.

In [ ]:
from tqdm.autonotebook import tqdm

In [ ]:
import os
root = '/kaggle/input/aptos2019-blindness-detection/'
train_img_path = os.path.join(root,'train_images')
train_path = os.path.join(root,'train.csv')
test_img_path = os.path.join(root,'test_images')
test_path = os.path.join(root,'test.csv')

In [ ]:
import pandas as pd
train_data = pd.read_csv(train_path)
train_data.head()

In [ ]:
train_data.hist()

In [ ]:
n_classes = train_data.diagnosis.nunique()

In [ ]:
classes = train_data.diagnosis.unique()

In [ ]:
train_data.diagnosis.value_counts()

In [ ]:
train_data['id_code'] = train_data['id_code'].apply(lambda x:x+'.png')
train_data['diagnosis'] = train_data['diagnosis'].apply(lambda x:str(x))
train_data.head()

In [ ]:
train_data.dtypes

# Model

In [ ]:
# define the size of our input data
sz=224

In [ ]:
from keras import layers
from keras import models

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(sz, sz, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(n_classes, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
from keras import optimizers


'''
opt = Adam(lr=1e-4)
model.compile(optimizer=opt,loss='categorical_crossentropy', metrics=['accuracy'])
'''
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

# Train

In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
#import keras
#from keras.models import Sequential, Model 
from keras.preprocessing.image import ImageDataGenerator
#from keras.applications.vgg16 import VGG16, preprocess_input
#from keras.layers import Dropout, Flatten,Dense


import numpy as np
import os
from matplotlib import image,patches,patheffects
import matplotlib.pyplot as plt
from PIL import Image

# our batch size
bs=32

# No need for preprocess_fn as it will be handled in target_sz of the generator

import cv2
def preprocess_input(img):
    #return img/255#cv2.resize(img, (sz,sz))
    return img.astype('float32')/255

# preprocess_input is for VGG16 in our case
train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input,
                                   rotation_range=20,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True) 

valid_datagen = ImageDataGenerator(preprocessing_function=preprocess_input) 


train_batches = train_datagen.flow_from_dataframe(train_df, # The df
                                                  train_img_path, # Place on desk
                                                  x_col='id_code', # The column to get x
                                                  y_col='diagnosis', # The column to get y
                                                  #has_ext=True, 
                                                  target_size=(sz, sz), 
                                                  color_mode='rgb', 
                                                  classes=None, 
                                                  class_mode='categorical', 
                                                  batch_size=bs, 
                                                  shuffle=True)



valid_batches = valid_datagen.flow_from_dataframe(val_df, 
                                                  train_img_path, 
                                                  x_col='id_code', 
                                                  y_col='diagnosis', 
                                                  #has_ext=True, 
                                                  target_size=(sz, sz), 
                                                  color_mode='rgb', 
                                                  classes=None,#list(train_batches.class_indices),#classes, 
                                                  class_mode='categorical', 
                                                  batch_size=bs, 
                                                  shuffle=False)

NbClasses = len(train_batches.class_indices)

In [ ]:
from sklearn.metrics import cohen_kappa_score
from keras.callbacks import Callback
class Kappa(Callback):
    
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        X_val, y_val = self.validation_data[0], self.validation_data[1]
        y_predict = np.asarray(model.predict(X_val))

        y_val = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)

        self._data.append({
            'val_kappa': cohen_kappa_score(y_val, y_predict),
        })
        return

    def get_data(self):
        return self._data

In [ ]:
import numpy as np


def confusion_matrix(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Returns the confusion matrix between rater's ratings
    """
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(rater_a + rater_b)
    if max_rating is None:
        max_rating = max(rater_a + rater_b)
    num_ratings = int(max_rating - min_rating + 1)
    conf_mat = [[0 for i in range(num_ratings)]
                for j in range(num_ratings)]
    for a, b in zip(rater_a, rater_b):
        conf_mat[a - min_rating][b - min_rating] += 1
    return conf_mat


def histogram(ratings, min_rating=None, max_rating=None):
    """
    Returns the counts of each type of rating that a rater made
    """
    if min_rating is None:
        min_rating = min(ratings)
    if max_rating is None:
        max_rating = max(ratings)
    num_ratings = int(max_rating - min_rating + 1)
    hist_ratings = [0 for x in range(num_ratings)]
    for r in ratings:
        hist_ratings[r - min_rating] += 1
    return hist_ratings


def quadratic_weighted_kappa(rater_a, rater_b, min_rating=None, max_rating=None):
    """
    Calculates the quadratic weighted kappa
    quadratic_weighted_kappa calculates the quadratic weighted kappa
    value, which is a measure of inter-rater agreement between two raters
    that provide discrete numeric ratings.  Potential values range from -1
    (representing complete disagreement) to 1 (representing complete
    agreement).  A kappa value of 0 is expected if all agreement is due to
    chance.

    quadratic_weighted_kappa(rater_a, rater_b), where rater_a and rater_b
    each correspond to a list of integer ratings.  These lists must have the
    same length.

    The ratings should be integers, and it is assumed that they contain
    the complete range of possible ratings.

    quadratic_weighted_kappa(X, min_rating, max_rating), where min_rating
    is the minimum possible rating, and max_rating is the maximum possible
    rating
    """
    rater_a = np.array(rater_a, dtype=int)
    rater_b = np.array(rater_b, dtype=int)
    assert(len(rater_a) == len(rater_b))
    if min_rating is None:
        min_rating = min(min(rater_a), min(rater_b))
    if max_rating is None:
        max_rating = max(max(rater_a), max(rater_b))
    conf_mat = confusion_matrix(rater_a, rater_b,
                                min_rating, max_rating)
    num_ratings = len(conf_mat)
    num_scored_items = float(len(rater_a))

    hist_rater_a = histogram(rater_a, min_rating, max_rating)
    hist_rater_b = histogram(rater_b, min_rating, max_rating)

    numerator = 0.0
    denominator = 0.0

    for i in range(num_ratings):
        for j in range(num_ratings):
            expected_count = (hist_rater_a[i] * hist_rater_b[j]
                              / num_scored_items)
            d = pow(i - j, 2.0) / pow(num_ratings - 1, 2.0)
            numerator += d * conf_mat[i][j] / num_scored_items
            denominator += d * expected_count / num_scored_items

    return 1.0 - numerator / denominator

In [ ]:
X_val = []
y_val = []
s = 0
validation_steps = valid_batches.n // valid_batches.batch_size
print(validation_steps)
for b in tqdm(valid_batches):
#for s in validation_steps:
    #b = next(valid_batches)
    #print(s)
    if s > validation_steps: # to prevent looping forever
        break
    s+=1
    X_val.append(b[0])
    y_val.append(b[1])
X_val = np.concatenate(X_val)    
y_val = np.concatenate(y_val) 
   

In [ ]:
print(X_val.shape)
print(y_val.shape)

In [ ]:
from sklearn.metrics import cohen_kappa_score

class QWKP(Callback):
    
    def on_train_begin(self, logs={}):
        self._data = []

    def on_epoch_end(self, batch, logs={}):
        #X_val, y_val = self.validation_data[0], self.validation_data[1]

        
        y_predict = np.asarray(model.predict(X_val))

        y_val_ = np.argmax(y_val, axis=1)
        y_predict = np.argmax(y_predict, axis=1)
        qwkp =  cohen_kappa_score(y_val_, y_predict)#quadratic_weighted_kappa(y_val_, y_predict)
        print('Validation qwkp: ', str(qwkp))
        self._data.append({
            'val_kappa':qwkp,
        })
        return

    def get_data(self):
        return self._data

In [ ]:
epochs = 2

history = model.fit_generator(train_batches,
                              steps_per_epoch = train_batches.n // train_batches.batch_size,
                              epochs=epochs,
                              validation_data=valid_batches,
                              validation_steps = valid_batches.n // valid_batches.batch_size,
                              callbacks=[QWKP()])

In [ ]:
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# Visualization

In [ ]:
# Get a sample image
b = next(valid_batches)
img = b[0][5]
print(img.shape)
plt.imshow(img)

In [ ]:
np.max(img)

In [ ]:
np.argmax(b[1][5])

In [ ]:
x = np.expand_dims(img, 0)
preds = model.predict(x)
idx = np.argmax(preds[0])
idx

In [ ]:
model.summary()

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

In [ ]:
last_conv_layer.output_shape[-1]

In [ ]:
from keras import backend as K
# This is the index entry in the prediction vector
output = model.output[:, idx]

# The is the output feature map of the `block5_conv3` layer,
# the last convolutional layer in VGG16
last_conv_layer =model.layers[6] #model.get_layer('block5_conv3')

# This is the gradient of the "african elephant" class with regard to
# the output feature map of `block5_conv3`
grads = K.gradients(output, last_conv_layer.output)[0]

# This is a vector of shape (512,), where each entry
# is the mean intensity of the gradient over a specific feature map channel
pooled_grads = K.mean(grads, axis=(0, 1, 2))

# This function allows us to access the values of the quantities we just defined:
# `pooled_grads` and the output feature map of `block5_conv3`,
# given a sample image
iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

# These are the values of these two quantities, as Numpy arrays,
# given our sample image of two elephants
pooled_grads_value, conv_layer_output_value = iterate([x])

# We multiply each channel in the feature map array
# by "how important this channel is" with regard to the elephant class
#for i in range(128):
for i in range(last_conv_layer.output_shape[-1]):
    
    conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

# The channel-wise mean of the resulting feature map
# is our heatmap of class activation
heatmap = np.mean(conv_layer_output_value, axis=-1)

In [ ]:
heatmap.shape

In [ ]:
heatmap = np.maximum(heatmap, 0)
heatmap /= np.max(heatmap)
plt.matshow(heatmap)
plt.show()

In [ ]:
import cv2


# We resize the heatmap to have the same size as the original image
heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

# We convert the heatmap to RGB
heatmap = np.uint8(255 * heatmap)
#eatmap = np.uint8(heatmap)

# We apply the heatmap to the original image
heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

# 0.4 here is a heatmap intensity factor
superimposed_img = heatmap * 0.1 + img

# Save the image to disk
#cv2.imwrite('elephant_cam.jpg', superimposed_img)
plt.imshow(superimposed_img)

In [ ]:
from keras import backend as K
import cv2
def grad_CAM(model, layer, img):
    x = np.expand_dims(img, 0)
    preds = model.predict(x)
    idx = np.argmax(preds[0])

    
    # This is the index entry in the prediction vector
    output = model.output[:, idx]

    # The is the output feature map of the `block5_conv3` layer,
    # the last convolutional layer in VGG16
    last_conv_layer =model.layers[layer] #model.get_layer('block5_conv3')

    # This is the gradient of the "african elephant" class with regard to
    # the output feature map of `block5_conv3`
    grads = K.gradients(output, last_conv_layer.output)[0]

    # This is a vector of shape (512,), where each entry
    # is the mean intensity of the gradient over a specific feature map channel
    pooled_grads = K.mean(grads, axis=(0, 1, 2))

    # This function allows us to access the values of the quantities we just defined:
    # `pooled_grads` and the output feature map of `block5_conv3`,
    # given a sample image
    iterate = K.function([model.input], [pooled_grads, last_conv_layer.output[0]])

    # These are the values of these two quantities, as Numpy arrays,
    # given our sample image of two elephants
    pooled_grads_value, conv_layer_output_value = iterate([x])

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the elephant class
    for i in range(last_conv_layer.output_shape[-1]):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    # The channel-wise mean of the resulting feature map
    # is our heatmap of class activation
    heatmap = np.mean(conv_layer_output_value, axis=-1)
    
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap)
    #plt.matshow(heatmap)
    #plt.show()    
    heatmap_raw = heatmap
    
    # We resize the heatmap to have the same size as the original image
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # We convert the heatmap to RGB
    heatmap = np.uint8(255 * heatmap)
    #eatmap = np.uint8(heatmap)

    # We apply the heatmap to the original image
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)

    # 0.4 here is a heatmap intensity factor
    superimposed_img = heatmap * 0.1 + img

    # Save the image to disk
    #cv2.imwrite('elephant_cam.jpg', superimposed_img)
    #plt.imshow(superimposed_img)
    return superimposed_img, heatmap_raw

In [ ]:
# Get a sample image
b = next(valid_batches)
img_idx = 5
img = b[0][img_idx]
print(img.shape)
plt.imshow(img)
plt.show()
x = np.expand_dims(img, 0)
preds = model.predict(x)
idx = np.argmax(preds[0])
print('Pred class: ', str(idx), ' Correct class: ', str(np.argmax(b[1][img_idx])))
superimposed_img, heatmap = grad_CAM(model=model, layer=6, img=img)
plt.imshow(heatmap)
plt.show()
plt.imshow(superimposed_img)
plt.show()

In [ ]:
# Get a sample image
b = next(valid_batches)
for i in range(5):
    img_idx = i
    img = b[0][img_idx]
    #print(img.shape)
    plt.imshow(img)
    plt.show()
    x = np.expand_dims(img, 0)
    preds = model.predict(x)
    idx = np.argmax(preds[0])
    print('Pred class: ', str(idx), ' Correct class: ', str(np.argmax(b[1][img_idx])))
    superimposed_img, heatmap = grad_CAM(model=model, layer=6, img=img)
    plt.imshow(heatmap)
    plt.show()
    plt.imshow(superimposed_img)
    plt.show()

# From the rare class (severe)

In [ ]:
train_data = pd.read_csv(train_path)

In [ ]:
train_data[train_data.diagnosis==3].head()

In [ ]:
from keras.preprocessing import image
img = image.load_img(train_img_path + '/' + train_data[train_data.diagnosis==3].iloc[10].id_code + '.png', target_size=(sz,sz))

In [ ]:
img = preprocess_input(np.array(img))

In [ ]:
#plt.imshow(img.img_to_array())
#img = np.array(img)
#img = image.img_to_array(img)
print(img.shape)
plt.imshow(img)

In [ ]:

plt.imshow(img)
plt.show()
# = preprocess_input(image.img_to_array(img))
x = np.expand_dims(img, axis=0)

preds = model.predict(x)
idx = np.argmax(preds[0])
print('Pred class: ', str(idx), ' Correct class: ', str(3))
superimposed_img, heatmap = grad_CAM(model=model, layer=6, img=image.img_to_array(img))
plt.imshow(heatmap)
plt.show()
plt.imshow(superimposed_img)
plt.show()

In [ ]:
from keras.preprocessing import image
for cl in range(4):
    print("Class: ", str(cl))
    for i in range(3):
        
        img = image.load_img(train_img_path + '/' + train_data[train_data.diagnosis==cl].iloc[i].id_code + '.png', target_size=(sz,sz))
        img = preprocess_input(np.array(img))
        
        plt.imshow(img)
        plt.show()
        # = preprocess_input(image.img_to_array(img))
        x = np.expand_dims(img, axis=0)

        preds = model.predict(x)
        idx = np.argmax(preds[0])
        print('Pred class: ', str(idx), ' Correct class: ', str(3))
        superimposed_img, heatmap = grad_CAM(model=model, layer=6, img=image.img_to_array(img))
        plt.imshow(heatmap)
        plt.show()
        plt.imshow(superimposed_img)
        plt.show()

The model is not able to recognize the rare class 3 (severe)
However, it seems to capture interesting abnormal areas in the feature map

Visually, the image looks "abnormal" in many areas.

It's usually confused the the "Moderate" class.

In [ ]:
from sklearn.metrics import confusion_matrix
y_predict = np.asarray(model.predict(X_val))
y_predict = np.argmax(y_predict, axis=1)

y_val_ = np.argmax(y_val, axis=1)

cm = confusion_matrix(y_val_, y_predict)

In [ ]:
import seaborn as sns
sns.heatmap(cm, annot=True, fmt="d")

As expected, the best performing classes are the dominant ones.

# Submission

In [ ]:
submission = pd.read_csv(os.path.join(root, 'sample_submission.csv'))
submission.head()

In [ ]:
submission['id_code'] = submission['id_code'].apply(lambda x:x+'.png')
#submission['diagnosis'] = submission['diagnosis'].apply(lambda x:str(x))
submission.head()

In [ ]:
test_df = pd.read_csv(test_path)

In [ ]:
test_df['id_code'] = test_df['id_code'].apply(lambda x:x+'.png')
#test_df['diagnosis'] = test_df['diagnosis'].apply(lambda x:str(x))
test_df.head()

In [ ]:
test_df.head()

In [ ]:
len(test_df)

In [ ]:
!ls {test_img_path}

In [ ]:
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
'''
submission.diagnosis = submission.diagnosis.apply(str)
test_batches = test_datagen.flow_from_dataframe(submission, 
                                                  test_img_path, 
                                                  x_col='id_code', 
                                                  y_col='diagnosis', 
                                                  #has_ext=True, 
                                                  target_size=(sz, sz), 
                                                  color_mode='rgb', 
                                                  classes=list(train_batches.class_indices),#classes, 
                                                  class_mode='categorical', #'input',
                                                  batch_size=bs, 
                                                  shuffle=False)
'''

test_batches = test_datagen.flow_from_dataframe(submission,#test_df, 
                                                  test_img_path, 
                                                  x_col='id_code', 
                                                  #y_col='diagnosis', 
                                                  #has_ext=True, 
                                                  target_size=(sz, sz), 
                                                  color_mode='rgb', 
                                                  #classes=list(train_batches.class_indices),#classes, 
                                                  class_mode=None,#'categorical', #'input',
                                                  batch_size=1, 
                                                  shuffle=False)

preds = []
'''
i = 0
for batch in test_batches:

    #print(model.predict(batch))
    preds.append(model.predict(batch))#(np.argmax(model.predict(batch)))
    i+=1
    print(i)
    if i == len(test_df):
        break
'''        

for i in tqdm(range(len(test_df))):
    batch = next(test_batches)
    preds.append(np.argmax(model.predict(batch)))

In [ ]:
import numpy as np
submission['diagnosis'] = preds
submission.to_csv('submission.csv', index=False)
submission.head()